In [ ]:
# https://scikit-image.org/docs/dev/auto_examples/applications/plot_human_mitosis.html

import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage as ndi

from skimage import (
    color, feature, filters, measure, morphology, segmentation, util
)

# import SEM image and convert to binary
image = cv2.imread('C:/Users/.tiff')
image = image[:,:,0]

fig, ax = plt.subplots()
ax.imshow(image, cmap='gray')
plt.show()

In [ ]:
# Otsu thresholding
thresholds = filters.threshold_multiotsu(image, classes=3)
regions = np.digitize(image, bins=thresholds)

fig, ax = plt.subplots(ncols=2, figsize=(10, 5))
ax[0].imshow(image)
ax[0].set_title('Original')
ax[0].axis('off')
ax[1].imshow(regions)
ax[1].set_title('Multi-Otsu thresholding')
ax[1].axis('off')
plt.show()

In [ ]:
# include pixels above value 1. background pixels are 0. 
cells = image > thresholds[1]

In [ ]:
# watershed segmentation 
distance = ndi.distance_transform_edt(cells)

local_max_coords = feature.peak_local_max(distance, min_distance=5)
local_max_mask = np.zeros(distance.shape, dtype=bool)
local_max_mask[tuple(local_max_coords.T)] = True
markers = measure.label(local_max_mask)

segmented_cells = segmentation.watershed(-distance, markers, mask=cells)

In [ ]:
# visualize segmentation
fig, ax = plt.subplots(ncols=2, figsize=(10, 5))
ax[0].imshow(cells, cmap='gray')
ax[0].set_title('Overlapping cells')
ax[0].axis('off')
ax[1].imshow(color.label2rgb(segmented_cells, bg_label=0))
ax[1].set_title('Segmented cells')
ax[1].axis('off')
plt.show()

In [ ]:
# visualize number of lamellipodia
color_labels = color.label2rgb(segmented_cells, image, alpha=0.4, bg_label=0)

fig, ax = plt.subplots(figsize=(5, 5))
ax.imshow(color_labels)
ax.set_title('Segmentation result over raw image')
plt.show()

In [ ]:
# number of lamellipodia
print(segmented_cells.max())